In [ ]:
!mkdir banners

In [ ]:
!pip install psd-tools

In [ ]:
import os
import pandas as pd
from psd_tools import PSDImage

In [ ]:
folder_path = "/content/banners"
all_data = []
kv_data = {}  # Información de los Key Visuals (KV)

In [ ]:
# Asignar tipo de elemento a cada capa
def assign_element_type(layer_name):
    name = layer_name.lower()
    if "fixed" in name or "forced" in name or "locked" in name:
        return "FIXED"
    elif "cta" in name or "button" in name:
        return "CTA"
    elif "product" in name or "sku" in name:
        return "PRODUCT"
    elif "logo" in name:
        return "LOGO"
    elif "legal" in name:
        return "LEGAL"
    elif "copy" in name or "text" in name or "subheading" in name:
        return "TEXT"
    elif "background" in name:
        return "BACKGROUND"
    else:
        return "OTHER"

In [ ]:
# Asignar prioridad según tipo de elemento
def assign_priority(element_type):
    if element_type == "FIXED":
        return 3
    elif element_type in ["CTA", "PRODUCT", "LOGO", "LEGAL", "TEXT"]:
        return 2
    elif element_type == "BACKGROUND":
        return -1
    else:
        return 0


In [ ]:
# Extraer capas de un PSD
def extract_layers(psd, filename):
    elements = []
    canvas_width = psd.width
    canvas_height = psd.height

    for i, layer in enumerate(psd):
        if not layer.is_group() and layer.bbox is not None:
            left, top, right, bottom = layer.bbox
            width = right - left
            height = bottom - top
            element_type = assign_element_type(layer.name)
            priority = assign_priority(element_type)

            elements.append({
                'file': filename,
                'layer_name': layer.name,
                'type': element_type,
                'priority': priority,
                'x': left,
                'y': top,
                'width': width,
                'height': height,
                'z_index': i,
                'canvas_width': canvas_width,
                'canvas_height': canvas_height
            })
    return elements

In [ ]:
# Obtener nombre base (sin dimensiones ni "KV")
def get_base_name(filename):
    parts = filename.replace(".psd", "").split("_")
    last_part = parts[-1].lower()
    if last_part == "kv" or "x" in last_part:
        parts = parts[:-1]
    return "_".join(parts)

In [ ]:
# -------- PRIMERA PASADA: PROCESAR KVs -------- #

for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".psd") and "kv" in filename.lower():
        filepath = os.path.join(folder_path, filename)
        psd = PSDImage.open(filepath)
        base_name = get_base_name(filename)

        elements = extract_layers(psd, filename)

        kv_data[base_name] = {
            'canvas_width': psd.width,
            'canvas_height': psd.height,
            'elements': elements
        }

In [ ]:
# -------- SEGUNDA PASADA: PROCESAR BANNERS -------- #

for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".psd") and "kv" not in filename.lower():
        filepath = os.path.join(folder_path, filename)
        psd = PSDImage.open(filepath)
        base_name = get_base_name(filename)

        elements = extract_layers(psd, filename)

        kv_info = kv_data.get(base_name)
        kv_canvas_width = kv_canvas_height = None
        kv_x = kv_y = kv_width = kv_height = kv_z = None
        text_x = text_y = text_w = text_h = text_z = None

        # Buscar info del KV
        if kv_info:
            kv_canvas_width = kv_info['canvas_width']
            kv_canvas_height = kv_info['canvas_height']
            for el in kv_info['elements']:
                if el['type'] in ["PRODUCT", "TEXT"]:
                    kv_x = el['x']
                    kv_y = el['y']
                    kv_width = el['width']
                    kv_height = el['height']
                    kv_z = el['z_index']
                    break

        # Buscar capa de texto en el banner
        for el in elements:
            if el['type'] == "TEXT":
                text_x = el['x']
                text_y = el['y']
                text_w = el['width']
                text_h = el['height']
                text_z = el['z_index']
                break

        # Agregar datos a la lista
        for el in elements:
            all_data.append({
                **el,
                'kv_canvas_width': kv_canvas_width,
                'kv_canvas_height': kv_canvas_height,
                'kv_x': kv_x,
                'kv_y': kv_y,
                'kv_width': kv_width,
                'kv_height': kv_height,
                'kv_z_index': kv_z,
                'text_x': text_x,
                'text_y': text_y,
                'text_width': text_w,
                'text_height': text_h,
                'text_z_index': text_z
            })

In [ ]:
df = pd.DataFrame(all_data)
df.to_csv("banners_metadata.csv", index=False)
print(df.head())

                file   layer_name        type  priority   x    y  width  \
0  Bubly_160X600.psd   BACKGROUND  BACKGROUND        -1   0    0    160   
1  Bubly_160X600.psd         SKUs     PRODUCT         2   0  224    160   
2  Bubly_160X600.psd         COPY        TEXT         2  12   85    139   
3  Bubly_160X600.psd          CTA         CTA         2  12  522    139   
4  Bubly_160X600.psd  SUB HEADING       OTHER         0  14  406    132   

   height  z_index  canvas_width  ...   kv_x   kv_y  kv_width  kv_height  \
0     600        0           160  ...  172.0  219.0    1361.0      162.0   
1     334        1           160  ...  172.0  219.0    1361.0      162.0   
2      66        2           160  ...  172.0  219.0    1361.0      162.0   
3      37        3           160  ...  172.0  219.0    1361.0      162.0   
4      63        4           160  ...  172.0  219.0    1361.0      162.0   

   kv_z_index  text_x  text_y  text_width  text_height  text_z_index  
0         2.0      12